# SQL Line-by-Line Comparison Widget

This notebook implements a specialized visualization for comparing SQL queries line by line, focusing on associating similar clauses, conditions, and statements between gold (reference) and predicted SQL queries. 

The visualization pipeline:
1. Parses SQL queries into meaningful "lines" (clauses, conditions, etc.)
2. Calculates similarity between lines from predicted and gold queries
3. Associates lines from both queries based on similarity thresholds
4. Visualizes the matching with appropriate highlighting (green for matched, red for unmatched)

In [1]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import sqlparse
import difflib
from pygments import highlight
from pygments.lexers import SqlLexer
from pygments.formatters import HtmlFormatter
import json
import ast
from datetime import datetime
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources (required for text processing)
nltk.download('punkt')
nltk.download('stopwords')

# Import the utility functions for evaluation
from utils.eval_utils import load_evaluation_results_from_dir

[nltk_data] Downloading package punkt to /home/jespejo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jespejo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load evaluation results
save_path = "../results"

evaluation_results = load_evaluation_results_from_dir(save_path=save_path)
evaluation_results_train = load_evaluation_results_from_dir(save_path=save_path, dataset="txt2sql_alerce_train_v4_0")
evaluation_results_test = load_evaluation_results_from_dir(save_path=save_path, dataset="txt2sql_alerce_test_v4_0")

# Load datasets
try:
    db_train = pd.read_csv('data/txt2sql_alerce_train_v4_0.csv')
    db_test = pd.read_csv('data/txt2sql_alerce_test_v4_0.csv')
    print(f"Loaded training dataset with {len(db_train)} examples")
    print(f"Loaded test dataset with {len(db_test)} examples")
    
    # Print column names for debugging
    print("\nTraining dataset columns:")
    print(db_train.columns.tolist())
    print("\nTest dataset columns:")
    print(db_test.columns.tolist())
    
    # Show a sample row to understand the structure
    if len(db_train) > 0:
        print("\nSample row from training dataset:")
        print(db_train.iloc[0].to_dict())
        
except Exception as e:
    print(f"Warning: Could not load datasets: {e}")
    db_train = None
    db_test = None

Skipping Qwen2.5-Coder-3B/alerce_train_direct_v8 as config.json does not exist.
Skipping gpt-4.1-mini-2025-04-14/alerce_train_direct_v8 as config.json does not exist.
Skipping prem-1B-SQL/alerce_train_direct_v8 as config.json does not exist.
Evaluation results file not found for Qwen2.5-1.5B-Instruct/alerce_dummy_direct_v8. Skipping...
Skipping Qwen2.5-Coder-3B/alerce_train_direct_v8 as config.json does not exist.
Skipping gpt-4.1-mini-2025-04-14/alerce_train_direct_v8 as config.json does not exist.
Skipping prem-1B-SQL/alerce_train_direct_v8 as config.json does not exist.
Skipping Qwen2.5-Coder-3B/alerce_train_direct_v8 as config.json does not exist.
Skipping gpt-4.1-mini-2025-04-14/alerce_train_direct_v8 as config.json does not exist.
Skipping prem-1B-SQL/alerce_train_direct_v8 as config.json does not exist.
Loaded training dataset with 58 examples
Loaded test dataset with 52 examples

Training dataset columns:
['req_id', 'request', 'table_info', 'external_knowledge', 'domain_knowled

In [3]:
# Process evaluation results to a format usable by the widget
models = list(evaluation_results.keys())

self_corr = True
self_corr_key = 'self_corrected' if self_corr else 'corrected'

# Initialize result dictionary for the widget
results_dict = {}

for model in models:
    experiments = list(evaluation_results[model].keys())
    for experiment in experiments:
        try:
            exp_label = f"{model}-{experiment}"
            results = evaluation_results[model][experiment][self_corr_key]['detailed_results']
            results_dict[exp_label] = results
            print(f"Added {len(results)} results for {exp_label}")
        except KeyError as e:
            print(f"Warning: Missing data for {model} - {experiment}: {e}")

Added 104 results for claude-3-5-sonnet-20241022-alerce_test_sbs_v4
Added 104 results for claude-3-5-sonnet-20241022-alerce_test_direct_v8
Added 104 results for claude-3-7-sonnet-20250219-alerce_test_sbs_v4
Added 104 results for claude-3-7-sonnet-20250219-alerce_test_direct_v8
Added 116 results for gpt-4.1-nano-2025-04-14-alerce_train_direct_v10
Added 104 results for gpt-4.1-nano-2025-04-14-alerce_test_sbs_v4
Added 20 results for gpt-4.1-nano-2025-04-14-alerce_train_dummy
Added 116 results for gpt-4.1-nano-2025-04-14-alerce_train_direct_v9
Added 20 results for gpt-4.1-nano-2025-04-14-alerce_direct_dummy_prompts
Added 104 results for gpt-4.1-nano-2025-04-14-alerce_test_direct_v8
Added 116 results for gpt-4.1-nano-2025-04-14-alerce_train_sbscot_v0
Added 232 results for gpt-4.1-nano-2025-04-14-alerce_train_direct_v8
Added 20 results for gpt-4.1-nano-2025-04-14-alerce_sbs_dummy
Added 20 results for gpt-4.1-nano-2025-04-14-alerce_direct_dummy
Added 20 results for gpt-4.1-nano-2025-04-14-ale

In [4]:
# Install required packages for word embeddings
try:
    import sentence_transformers
except ImportError:
    print("Installing sentence-transformers...")
    !pip install sentence-transformers
    import sentence_transformers
    print("sentence-transformers installed successfully")

# Import necessary modules for word embeddings
from sentence_transformers import SentenceTransformer

# Global variable to hold the sentence transformer model
sentence_model = None

In [5]:
# Initialize the sentence transformer model for embeddings
# Using a lighter model for better performance
sentence_model = None

def initialize_sentence_model():
    """Initialize the sentence transformer model if not already loaded"""
    global sentence_model
    if sentence_model is None:
        try:
            # Using a smaller model for better performance
            sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("Loaded sentence-transformer model: all-MiniLM-L6-v2")
        except Exception as e:
            print(f"Error loading sentence-transformer model: {e}")
            print("Word embedding similarity method will not be available")

def calculate_similarity_with_method(line1, line2, method='tfidf'):
    """
    Calculate similarity between two SQL query lines using the specified method
    
    Parameters:
    -----------
    line1 : str
        The first line (SQL query fragment to compare)
    line2 : str
        The second line (SQL query fragment to compare)
    method : str
        The method to use for similarity calculation:
        - 'tfidf': Uses TF-IDF vectorization and cosine similarity (default)
        - 'jaccard': Uses Jaccard similarity (set-based comparison)
        - 'embedding': Uses neural word embeddings via sentence-transformers
        
    Returns:
    --------
    float
        Similarity score between 0 and 1
        
    Notes:
    ------
    This function handles special cases like:
    - Table aliases (e.g., "t.column" vs "column")
    - IN clauses with the same values but different column references
    - Empty or null input values
    
    Each similarity method has different characteristics:
    - TF-IDF: Fast, works well for keyword matching
    - Jaccard: Good for exact token overlap, less sensitive to order
    - Embedding: Best for semantic similarity, captures meaning even with different words
    """
    # Ensure both lines are strings
    if line1 is None:
        line1 = ""
    if line2 is None:
        line2 = ""
    
    if not isinstance(line1, str):
        line1 = str(line1)
    if not isinstance(line2, str):
        line2 = str(line2)
    
    # Special case: Direct equality check after alias normalization
    # This ensures that 'column = value' and 'table.column = value' are matched with high similarity
    normalized_line1 = re.sub(r'([a-z0-9_]+)\.([a-z0-9_]+)', r'\2', line1, flags=re.IGNORECASE)
    normalized_line2 = re.sub(r'([a-z0-9_]+)\.([a-z0-9_]+)', r'\2', line2, flags=re.IGNORECASE)
    
    # If the normalized lines are identical, return a very high similarity score
    extra_similarity = 0.0  # Extra similarity for IN clauses with identical values
    if normalized_line1.lower() == normalized_line2.lower():
        extra_similarity += 0.6
    
    # Handle table aliases and preprocessing regardless of method
    def preprocess_text(text):
        try:
            # Convert to lowercase
            text = text.lower()
            
            # Handle table aliases (e.g., "t.column" vs "table.column" vs "column")
            # Replace patterns like "x.column" with just "column"
            text = re.sub(r'([a-z0-9_]+)\.([a-z0-9_]+)', r'\2', text)
            
            # Now tokenize
            tokens = word_tokenize(text)
            
            # Remove SQL keywords, punctuation, and stopwords
            sql_keywords = ['select', 'from', 'where', 'group', 'by', 'having', 'order', 'limit',
                           'join', 'inner', 'outer', 'left', 'right', 'on', 'and', 'or', 'as', 'in']
            
            # Get stopwords or use empty list if not available
            try:
                stop_words = stopwords.words('english')
            except LookupError:
                print("Warning: Stopwords not available, using empty list")
                stop_words = []
                
            tokens = [token for token in tokens if token.isalnum() and token not in sql_keywords
                     and token not in stop_words]
            return ' '.join(tokens), tokens
        except Exception as e:
            print(f"Warning: Error preprocessing text: {e}")
            # Return a safe value
            return text.lower(), text.lower().split()
    
    # Special handling for IN clauses with the same values but different table aliases
    in_pattern = re.compile(r'(.*)\s+in\s+\((.*)\)', re.IGNORECASE)
    match1 = in_pattern.match(line1)
    match2 = in_pattern.match(line2)
    
    if match1 and match2:
        # If both are IN clauses, compare their values
        values1 = [v.strip().strip("'\"") for v in match1.group(2).split(',')]
        values2 = [v.strip().strip("'\"") for v in match2.group(2).split(',')]
        
        # If the values are identical, increase the similarity
        if sorted(values1) == sorted(values2):
            # Extract column names (removing table aliases)
            col1 = re.sub(r'([a-z0-9_]+)\.([a-z0-9_]+)', r'\2', match1.group(1), flags=re.IGNORECASE).strip()
            col2 = re.sub(r'([a-z0-9_]+)\.([a-z0-9_]+)', r'\2', match2.group(1), flags=re.IGNORECASE).strip()
            
            # If the column names match after removing aliases, add extra similarity
            if col1.lower() == col2.lower():
                extra_similarity += 0.5

    # Special handling for comparison operators (=, >, <, etc.) with different table aliases
    # This handles cases like "column = value" vs "table.column = value"
    comp_pattern = re.compile(r'(.*?)\s*(=|<>|!=|>|<|>=|<=)\s*(.*)', re.IGNORECASE)
    comp_match1 = comp_pattern.match(line1)
    comp_match2 = comp_pattern.match(line2)
    
    if comp_match1 and comp_match2:
        # If both are comparison expressions with the same operator
        if comp_match1.group(2) == comp_match2.group(2):
            # Check if the values being compared are the same
            val1 = comp_match1.group(3).strip().strip("'\"")
            val2 = comp_match2.group(3).strip().strip("'\"")
            
            if val1.lower() == val2.lower():
                # Extract column names (removing table aliases)
                col1 = re.sub(r'([a-z0-9_]+)\.([a-z0-9_]+)', r'\2', comp_match1.group(1), flags=re.IGNORECASE).strip()
                col2 = re.sub(r'([a-z0-9_]+)\.([a-z0-9_]+)', r'\2', comp_match2.group(1), flags=re.IGNORECASE).strip()
                
                # If the column names match after removing aliases, add extra similarity
                if col1.lower() == col2.lower():
                    extra_similarity += 0.5
    
    # Preprocess both lines
    processed_line1, tokens1 = preprocess_text(line1)
    processed_line2, tokens2 = preprocess_text(line2)
    
    # If either line is empty after preprocessing, return 0
    if not processed_line1 or not processed_line2:
        processed_line1 = line1.lower()
        processed_line2 = line2.lower()            # Calculate similarity based on the chosen method
    if method == 'tfidf':
        # TF-IDF vectorization and cosine similarity
        try:
            vectorizer = TfidfVectorizer(lowercase=True, analyzer='word', 
                                        token_pattern=r'[a-zA-Z0-9_\.\*]+',
                                        ngram_range=(1, 2))
            
            tfidf_matrix = vectorizer.fit_transform([processed_line1, processed_line2])
            similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
            similarity = min(1.0, similarity + extra_similarity)  # Cap at 1.0
            return similarity
        except Exception as e:
            print(f"Warning: Error calculating TF-IDF similarity: {e}")
            return 0
            
    elif method == 'jaccard':
        # Jaccard similarity (set-based)
        try:
            set1 = set(tokens1)
            set2 = set(tokens2)
            
            if not set1 or not set2:
                return 0
                
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            
            similarity = intersection / union if union > 0 else 0
            similarity = min(1.0, similarity + extra_similarity)  # Cap at 1.0
            return similarity
        except Exception as e:
            print(f"Warning: Error calculating Jaccard similarity: {e}")
            return 0
            
    elif method == 'embedding':
        # Word embedding similarity using sentence transformers
        try:
            # Initialize model if not already done
            if sentence_model is None:
                initialize_sentence_model()
                
            if sentence_model is None:
                # Fall back to TF-IDF if model initialization failed
                print("Warning: Falling back to TF-IDF due to missing embedding model")
                return calculate_similarity_with_method(line1, line2, 'tfidf')
                
            # Generate embeddings
            embedding1 = sentence_model.encode([processed_line1])[0]
            embedding2 = sentence_model.encode([processed_line2])[0]
            
            # Calculate cosine similarity between embeddings
            similarity = cosine_similarity([embedding1], [embedding2])[0][0]
            similarity = min(1.0, similarity + extra_similarity)  # Cap at 1.0
            return similarity
        except Exception as e:
            print(f"Warning: Error calculating embedding similarity: {e}")
            # Fall back to TF-IDF
            return calculate_similarity_with_method(line1, line2, 'tfidf')
    
    else:
        print(f"Warning: Unknown similarity method '{method}', falling back to TF-IDF")
        return calculate_similarity_with_method(line1, line2, 'tfidf')

In [6]:
def format_sql_query(query):
    """
    Format a SQL query for better readability.
    
    Parameters:
    -----------
    query : str
        The SQL query to format
        
    Returns:
    --------
    str
        The formatted SQL query
    """
    if query is None:
        return ""
    
    # Ensure query is a string
    if not isinstance(query, str):
        query = str(query)
    
    # Clean up the query
    query = query.strip()
    
    # Remove extra quotes at the beginning and end if present
    if (query.startswith('"') and query.endswith('"')) or \
       (query.startswith("'") and query.endswith("'")):
        query = query[1:-1]
    
    # If empty after cleanup, return empty string
    if not query:
        return ""
    
    try:
        # Format using sqlparse
        formatted_query = sqlparse.format(
            query,
            reindent=True,
            keyword_case='upper',
            strip_comments=True
        )
        return formatted_query
    except Exception as e:
        print(f"Warning: Could not format SQL query: {e}")
        # Return the original query if formatting fails
        return query

In [7]:
def highlight_sql(query):
    """
    Apply syntax highlighting to SQL code.
    
    Parameters:
    -----------
    query : str
        The SQL query to highlight
        
    Returns:
    --------
    str
        HTML string with highlighted SQL
    """
    if not query:
        return ""
    
    try:
        # Use Pygments for syntax highlighting
        highlighted = highlight(
            query,
            SqlLexer(),
            HtmlFormatter(style='default', noclasses=True)
        )
        return highlighted
    except Exception as e:
        print(f"Warning: Could not highlight SQL query: {e}")
        return f"<pre>{query}</pre>"

In [8]:
def split_where_conditions(where_content):
    """
    Split WHERE clause conditions into individual conditions while respecting
    parentheses, quotes, and subqueries.
    
    Parameters:
    -----------
    where_content : str
        The WHERE clause content to split
        
    Returns:
    --------
    list
        A list of individual conditions
    """
    from sql_line_comparison_widget_helpers import handle_complex_join_conditions
    
    # We can reuse the logic from handle_complex_join_conditions since the parsing is similar
    return handle_complex_join_conditions(where_content)

def parse_sql_into_lines(sql_query):
    """
    Parse a SQL query into meaningful lines (clauses, conditions, etc.) with improved support for subqueries
    
    Parameters:
    -----------
    sql_query : str
        The SQL query to parse
        
    Returns:
    --------
    list
        A list of tuples (line_type, line_content) representing the parsed lines
    """
    # Import the helper functions for JOIN and ON clause handling
    from sql_line_comparison_widget_helpers import split_join_clause, extract_on_clauses_from_joins, handle_complex_join_conditions
    
    # Basic validation and cleanup
    if sql_query is None:
        print("Warning: SQL query is None")
        return [('EMPTY', '')]
        
    # Strip whitespace and handle empty queries
    sql_query = sql_query.strip() if isinstance(sql_query, str) else ""
    if not sql_query:
        print("Warning: SQL query is empty")
        return [('EMPTY', '')]
    
    # Format the SQL query
    formatted_sql = format_sql_query(sql_query)
    
    # Initialize the result list
    parsed_lines = []
    
    # Handle very simple queries or non-standard SQL
    if not re.search(r'\bSELECT\b', formatted_sql, re.IGNORECASE):
        print("No SELECT statement found, treating as custom SQL")
        # Split by semicolons for multiple statements
        statements = formatted_sql.split(';')
        for stmt in statements:
            stmt = stmt.strip()
            if stmt:
                parsed_lines.append(('SQL Statement', stmt))
        
        if parsed_lines:
            return parsed_lines
        else:
            return [('FULL QUERY', formatted_sql)]
    
    # Extract subqueries first to handle them separately
    subqueries = extract_subqueries(formatted_sql)
    
    # Replace subqueries with placeholders in the main query
    modified_sql = formatted_sql
    subquery_placeholders = {}
    
    for i, (subquery_type, subquery_content) in enumerate(subqueries):
        placeholder = f"__SUBQUERY_{i}__"
        # Generate a unique signature for this subquery to ensure precise replacements
        subquery_signature = f"({subquery_content})"
        
        # For more complex replacements, we'll use a more precise method
        # Find the exact position of this subquery in the SQL text
        pos = modified_sql.find(subquery_signature)
        if pos != -1:
            # Replace only at this specific position
            modified_sql = modified_sql[:pos] + placeholder + modified_sql[pos + len(subquery_signature):]
        else:
            # Fallback to regex replacement if direct find fails
            # Escape special characters in the subquery content for regex
            escaped_content = re.escape(subquery_content)
            # Match the subquery with its surrounding parentheses
            pattern = f"\\({escaped_content}\\)"
            # Replace only one occurrence at a time to avoid confusion with similar subqueries
            modified_sql = re.sub(pattern, placeholder, modified_sql, count=1)
        
        subquery_placeholders[placeholder] = (subquery_type, subquery_content)
    
    # Use regular expressions to extract main components
    # These patterns target specific SQL clauses and their contents
    sql_patterns = [
        # SELECT clause and items
        (r'SELECT\s+(DISTINCT\s+)?(.+?)(?=\s+FROM|\s*$)', 'SELECT'),
        
        # FROM clause and tables
        (r'FROM\s+(.+?)(?=\s+(?:WHERE|GROUP\s+BY|HAVING|ORDER\s+BY|LIMIT)|\s*$)', 'FROM'),
        
        # JOIN clauses with separate capture groups for table and ON condition
        (r'((?:LEFT|RIGHT|INNER|OUTER|CROSS|NATURAL)?\s*JOIN\s+.+?)(?:\s+ON\s+(.+?))?(?=\s+(?:LEFT|RIGHT|INNER|OUTER|CROSS|NATURAL)?\s*JOIN|\s+(?:WHERE|GROUP\s+BY|HAVING|ORDER\s+BY|LIMIT)|\s*$)', 'JOIN'),
        
        # WHERE clause and conditions
        (r'WHERE\s+(.+?)(?=\s+(?:GROUP\s+BY|HAVING|ORDER\s+BY|LIMIT)|\s*$)', 'WHERE'),
        
        # GROUP BY clause
        (r'GROUP\s+BY\s+(.+?)(?=\s+(?:HAVING|ORDER\s+BY|LIMIT)|\s*$)', 'GROUP BY'),
        
        # HAVING clause
        (r'HAVING\s+(.+?)(?=\s+(?:ORDER\s+BY|LIMIT)|\s*$)', 'HAVING'),
        
        # ORDER BY clause
        (r'ORDER\s+BY\s+(.+?)(?=\s+LIMIT|\s*$)', 'ORDER BY'),
        
        # LIMIT clause
        (r'LIMIT\s+(.+?)(?=\s*$)', 'LIMIT'),
    ]
    
    # Extract each component using the patterns
    for pattern, line_type in sql_patterns:
        try:
            matches = re.finditer(pattern, modified_sql, re.DOTALL | re.IGNORECASE)
            for match in matches:
                if line_type == 'SELECT':
                    # For SELECT, split into individual columns
                    if match.group(2):
                        # More robust column splitting - handles nested functions
                        select_items = []
                        current_item = ""
                        paren_count = 0
                        
                        for char in match.group(2) + ',':  # Add comma to process last item
                            if char == ',' and paren_count == 0:
                                select_items.append(current_item.strip())
                                current_item = ""
                            else:
                                current_item += char
                                if char == '(':
                                    paren_count += 1
                                elif char == ')':
                                    paren_count -= 1
                        
                        for item in select_items:
                            item = item.strip()
                            if item:
                                # Check if this item contains a subquery placeholder
                                subquery_found = False
                                for placeholder, (sq_type, sq_content) in subquery_placeholders.items():
                                    if placeholder in item:
                                        # Replace the placeholder with the original subquery
                                        real_item = item.replace(placeholder, f"({sq_content})")
                                        parsed_lines.append(('SELECT item with subquery', real_item))
                                        subquery_found = True
                                        break
                                
                                if not subquery_found:
                                    parsed_lines.append(('SELECT item', item))
                elif line_type == 'FROM':
                    # For FROM, extract each table
                    if match.group(1):
                        # Split the FROM clause by commas but respect subqueries
                        from_items = []
                        current_item = ""
                        paren_count = 0
                        
                        for char in match.group(1) + ',':  # Add comma to process last item
                            if char == ',' and paren_count == 0:
                                from_items.append(current_item.strip())
                                current_item = ""
                            else:
                                current_item += char
                                if char == '(':
                                    paren_count += 1
                                elif char == ')':
                                    paren_count -= 1
                        
                        for item in from_items:
                            item = item.strip()
                            if item and not re.search(r'^\s*JOIN\s+', item, re.IGNORECASE):
                                # Check if this item contains a subquery placeholder
                                subquery_found = False
                                for placeholder, (sq_type, sq_content) in subquery_placeholders.items():
                                    if placeholder in item:
                                        # Replace the placeholder with the original subquery
                                        real_item = item.replace(placeholder, f"({sq_content})")
                                        parsed_lines.append(('FROM subquery', real_item))
                                        subquery_found = True
                                        break
                                
                                if not subquery_found:
                                    # Check for aliased subqueries that might not have been caught
                                    if re.search(r'\)\s+[a-z0-9_]+\s*$', item, re.IGNORECASE):
                                        parsed_lines.append(('FROM subquery', item))
                                    else:
                                        parsed_lines.append(('FROM table', item))
                elif line_type == 'WHERE':
                    # For WHERE, split conditions by AND/OR but respect parentheses
                    if match.group(1):
                        where_content = match.group(1).strip()
                        
                        # Check if there are subquery placeholders first
                        has_placeholders = any(placeholder in where_content for placeholder in subquery_placeholders.keys())
                        
                        if has_placeholders:
                            # If we have placeholders, handle them first before splitting
                            processed_where = where_content
                            for placeholder, (sq_type, sq_content) in subquery_placeholders.items():
                                if placeholder in processed_where:
                                    # Replace the placeholder with the original subquery
                                    processed_where = processed_where.replace(placeholder, f"({sq_content})")
                            
                            # Now attempt to split by AND/OR with the replaced content
                            try:
                                # Split the WHERE conditions safely, respecting subqueries and parentheses
                                conditions = split_where_conditions(processed_where)
                                
                                # Add each condition separately
                                for condition in conditions:
                                    if condition.strip():
                                        parsed_lines.append(('WHERE condition with subquery', condition.strip()))
                            except Exception as e:
                                print(f"Error processing WHERE with subqueries: {e}")
                                # Fallback to adding the whole WHERE clause
                                parsed_lines.append(('WHERE with subquery', processed_where))
                        else:
                            # Split the WHERE conditions safely
                            conditions = split_where_conditions(where_content)
                            
                            # Add each condition separately
                            for condition in conditions:
                                if condition.strip():
                                    # Check for subquery placeholders that might have been missed
                                    subquery_found = False
                                    for placeholder, (sq_type, sq_content) in subquery_placeholders.items():
                                        if placeholder in condition:
                                            # Replace the placeholder with the original subquery
                                            real_condition = condition.replace(placeholder, f"({sq_content})")
                                            parsed_lines.append(('WHERE condition with subquery', real_condition.strip()))
                                            subquery_found = True
                                            break
                                    
                                    if not subquery_found:
                                        parsed_lines.append(('WHERE condition', condition.strip()))
                elif line_type == 'JOIN':
                    # For JOIN, handle the table part and ON condition separately
                    join_clause = match.group(1).strip()
                    on_condition = match.group(2).strip() if match.group(2) else ""
                    
                    # Use the helper function to separate JOIN table from ON condition
                    table_part, on_part = split_join_clause(join_clause)
                    
                    if table_part:
                        # Check for subquery placeholders in JOIN table part
                        subquery_found = False
                        for placeholder, (sq_type, sq_content) in subquery_placeholders.items():
                            if placeholder in table_part:
                                # Replace the placeholder with the original subquery
                                real_table = table_part.replace(placeholder, f"({sq_content})")
                                parsed_lines.append(('JOIN table subquery', real_table))
                                subquery_found = True
                                break
                        
                        if not subquery_found:
                            parsed_lines.append(('JOIN table', table_part))
                    
                    # Handle ON condition separately if it exists
                    if on_part:
                        # Check for subquery placeholders in ON conditions
                        subquery_found = False
                        for placeholder, (sq_type, sq_content) in subquery_placeholders.items():
                            if placeholder in on_part:
                                # Replace the placeholder with the original subquery
                                real_condition = on_part.replace(placeholder, f"({sq_content})")
                                parsed_lines.append(('JOIN ON condition with subquery', real_condition))
                                subquery_found = True
                                break
                        
                        if not subquery_found:
                            # Split the ON condition by AND/OR for better granularity
                            on_conditions = handle_complex_join_conditions(on_part)
                            for condition in on_conditions:
                                parsed_lines.append(('JOIN ON condition', condition))
                else:
                    # For other clauses, add the whole match
                    content = match.group(1).strip()
                    if content:
                        # Check for subquery placeholders in other clauses
                        subquery_found = False
                        for placeholder, (sq_type, sq_content) in subquery_placeholders.items():
                            if placeholder in content:
                                # Replace the placeholder with the original subquery
                                real_content = content.replace(placeholder, f"({sq_content})")
                                parsed_lines.append((f'{line_type} with subquery', real_content))
                                subquery_found = True
                                break
                        
                        if not subquery_found:
                            parsed_lines.append((line_type, content))
        except Exception as e:
            print(f"Error parsing SQL pattern {line_type}: {e}")
    
    # Add any subqueries that weren't used in the main query
    for subquery_type, subquery_content in subqueries:
        # Check if this subquery wasn't included already
        found = False
        for line_type, line_content in parsed_lines:
            if f"({subquery_content})" in line_content:
                found = True
                break
                
        if not found:
            # Add as a standalone subquery
            parsed_lines.append((f'Independent {subquery_type}', f"({subquery_content})"))
    
    # If we didn't parse anything meaningful, but we have SQL text, return the full query
    if not parsed_lines and formatted_sql:
        parsed_lines.append(('FULL QUERY', formatted_sql))
    
    # If still empty, mark as empty query
    if not parsed_lines:
        parsed_lines.append(('EMPTY', ''))
    
    # print(f"Parsed lines: {parsed_lines}")
    return parsed_lines

def extract_subqueries(sql_query):
    """
    Extract subqueries from a SQL query with enhanced detection capabilities
    
    Parameters:
    -----------
    sql_query : str
        The SQL query to extract subqueries from
        
    Returns:
    --------
    list
        A list of tuples (subquery_type, subquery_content)
    """
    subqueries = []
    
    # Helper function to scan for SELECT keyword
    def has_select_keyword(text, start_pos, scan_length=20):
        """Check if text contains SELECT keyword within scan_length chars from start_pos"""
        for j in range(start_pos, min(start_pos + scan_length, len(text))):
            if text[j:j+6].upper() == 'SELECT':
                return True
        return False
    
    # Helper function to extract subquery content
    def extract_subquery_content(text, start_pos):
        """
        Extract subquery content from text starting at start_pos
        Returns tuple: (end_position, subquery_content, is_subquery)
        """
        paren_count = 1  # Already encountered the opening parenthesis
        content = ""
        i = start_pos
        
        # Check if this might be a subquery by looking for SELECT
        if not has_select_keyword(text, i):
            # Skip to matching closing parenthesis for non-subquery
            while i < len(text) and paren_count > 0:
                if text[i] == '(':
                    paren_count += 1
                elif text[i] == ')':
                    paren_count -= 1
                if paren_count > 0:
                    content += text[i]
                i += 1
            return i, content, False
        
        # Extract the subquery content
        in_quotes = False
        quote_char = None
        
        while i < len(text) and paren_count > 0:
            # Handle quoted strings to avoid misinterpreting parentheses in strings
            if text[i] in ['"', "'"]:
                if not in_quotes:
                    in_quotes = True
                    quote_char = text[i]
                elif text[i] == quote_char:
                    in_quotes = False
                    quote_char = None
            
            # Only count parentheses when not in quotes
            if not in_quotes:
                if text[i] == '(':
                    paren_count += 1
                elif text[i] == ')':
                    paren_count -= 1
            
            # Only add to content if we're still inside the subquery
            # This ensures we don't include the closing parenthesis
            if paren_count > 0:
                content += text[i]
            i += 1
        
        # Return the exact position after the closing parenthesis
        return i, content.strip(), True
    
    # Process SQL string for common escape patterns
    # This helps avoid false positives in string literals
    def preprocess_sql(sql):
        # Replace string literals temporarily
        in_single_quote = False
        in_double_quote = False
        escaped = False
        processed_sql = ""
        
        for char in sql:
            if char == "'" and not in_double_quote and not escaped:
                in_single_quote = not in_single_quote
                processed_sql += char
            elif char == '"' and not in_single_quote and not escaped:
                in_double_quote = not in_double_quote
                processed_sql += char
            elif char == '\\' and not escaped:
                escaped = True
                processed_sql += char
            else:
                if escaped:
                    escaped = False
                processed_sql += ' ' if (in_single_quote or in_double_quote) and char == '(' else char
        
        return processed_sql
    
    # Preprocess SQL to handle string literals
    processed_sql = preprocess_sql(sql_query)
    
    # Look for subqueries in SELECT clauses with enhanced SELECT pattern
    # This pattern better handles complex SELECT structures
    select_pattern = re.compile(r'SELECT\s+(.*?)(?:\s+FROM|\s*$)', re.IGNORECASE | re.DOTALL)
    select_matches = select_pattern.finditer(processed_sql)
    
    for match in select_matches:
        select_content = match.group(1)
        # Find opening parentheses that might indicate a subquery
        i = 0
        while i < len(select_content):
            if select_content[i] == '(':
                i += 1  # Move past the opening parenthesis
                end_pos, subquery_content, is_subquery = extract_subquery_content(select_content, i)
                i = end_pos
                
                if is_subquery and subquery_content and 'SELECT' in subquery_content.upper():
                    subqueries.append(('SELECT subquery', subquery_content))
            else:
                i += 1
    
    # Look for subqueries in FROM clauses with enhanced pattern
    # This pattern better handles multiple FROM sources and JOINs
    from_pattern = re.compile(r'FROM\s+(.*?)(?:\s+WHERE|\s+GROUP\s+BY|\s+HAVING|\s+ORDER\s+BY|\s+LIMIT|\s+(?:LEFT|RIGHT|INNER|OUTER|CROSS|NATURAL)?\s*JOIN|\s*$)', re.IGNORECASE | re.DOTALL)
    from_matches = from_pattern.finditer(processed_sql)
    
    for match in from_matches:
        from_content = match.group(1)
        i = 0
        while i < len(from_content):
            if from_content[i] == '(':
                i += 1  # Move past the opening parenthesis
                end_pos, subquery_content, is_subquery = extract_subquery_content(from_content, i)
                i = end_pos
                
                if is_subquery and subquery_content and 'SELECT' in subquery_content.upper():
                    # Check if this is an aliased subquery
                    if i < len(from_content) and re.match(r'\s+(?:AS\s+)?[a-z0-9_]+', from_content[i:i+20], re.IGNORECASE):
                        subqueries.append(('FROM aliased subquery', subquery_content))
                    else:
                        subqueries.append(('FROM subquery', subquery_content))
            else:
                i += 1
    
    # Look for subqueries in WHERE clauses with enhanced pattern
    # This pattern better handles complex WHERE conditions
    where_pattern = re.compile(r'WHERE\s+(.*?)(?:\s+GROUP\s+BY|\s+HAVING|\s+ORDER\s+BY|\s+LIMIT|\s*$)', re.IGNORECASE | re.DOTALL)
    where_matches = where_pattern.finditer(processed_sql)
    
    for match in where_matches:
        where_content = match.group(1)
        i = 0
        while i < len(where_content):
            if where_content[i] == '(':
                # Check if this is a logical grouping or a subquery
                # Look ahead for SELECT or common subquery markers like IN, EXISTS
                in_subquery_context = False
                for j in range(max(0, i-10), i):
                    fragment = where_content[j:i].upper()
                    if any(marker in fragment for marker in [' IN ', ' EXISTS ', ' ANY ', ' ALL ', ' SOME ']):
                        in_subquery_context = True
                        break
                
                i += 1  # Move past the opening parenthesis
                end_pos, subquery_content, is_subquery = extract_subquery_content(where_content, i)
                i = end_pos
                
                if is_subquery and subquery_content and 'SELECT' in subquery_content.upper():
                    # Determine the specific type of WHERE subquery
                    if in_subquery_context:
                        subqueries.append(('WHERE operator subquery', subquery_content))
                    else:
                        subqueries.append(('WHERE subquery', subquery_content))
            else:
                i += 1
    
    # Look for subqueries in JOIN clauses with enhanced pattern
    # This pattern better separates table and condition parts
    join_pattern = re.compile(r'((?:LEFT|RIGHT|INNER|OUTER|CROSS|NATURAL)?\s*JOIN)\s+(.+?)(?:\s+ON\s+(.+?))?(?=\s+(?:LEFT|RIGHT|INNER|OUTER|CROSS|NATURAL)?\s*JOIN|\s+WHERE|\s+GROUP\s+BY|\s+HAVING|\s+ORDER\s+BY|\s+LIMIT|\s*$)', re.IGNORECASE | re.DOTALL)
    join_matches = join_pattern.finditer(processed_sql)
    
    for match in join_matches:
        join_type = match.group(1)
        join_table = match.group(2)
        join_condition = match.group(3) if match.group(3) else ""
        
        # Process the table part (check for subqueries)
        i = 0
        while i < len(join_table):
            if join_table[i] == '(':
                i += 1  # Move past the opening parenthesis
                end_pos, subquery_content, is_subquery = extract_subquery_content(join_table, i)
                i = end_pos
                
                if is_subquery and subquery_content and 'SELECT' in subquery_content.upper():
                    # Check if this is an aliased subquery
                    alias_match = re.search(r'\)\s+(?:AS\s+)?([a-z0-9_]+)', join_table[i-1:], re.IGNORECASE)
                    if alias_match:
                        subqueries.append(('JOIN aliased table subquery', subquery_content))
                    else:
                        subqueries.append(('JOIN table subquery', subquery_content))
            else:
                i += 1
        
        # Process the ON condition separately if it exists
        if join_condition:
            i = 0
            while i < len(join_condition):
                if join_condition[i] == '(':
                    i += 1  # Move past the opening parenthesis
                    end_pos, subquery_content, is_subquery = extract_subquery_content(join_condition, i)
                    i = end_pos
                    
                    if is_subquery and subquery_content and 'SELECT' in subquery_content.upper():
                        subqueries.append(('JOIN ON condition subquery', subquery_content))
                else:
                    i += 1
    
    # Look for subqueries in HAVING clauses
    having_pattern = re.compile(r'HAVING\s+(.*?)(?:\s+ORDER\s+BY|\s+LIMIT|\s*$)', re.IGNORECASE | re.DOTALL)
    for match in having_pattern.finditer(processed_sql):
        having_content = match.group(1)
        i = 0
        while i < len(having_content):
            if having_content[i] == '(':
                i += 1  # Move past the opening parenthesis
                end_pos, subquery_content, is_subquery = extract_subquery_content(having_content, i)
                i = end_pos
                
                if is_subquery and subquery_content and 'SELECT' in subquery_content.upper():
                    subqueries.append(('HAVING subquery', subquery_content))
            else:
                i += 1
    
    # Additional check for nested subqueries inside already extracted subqueries
    additional_subqueries = []
    for _, content in subqueries:
        # Check if this subquery contains other subqueries
        i = 0
        while i < len(content):
            if content[i] == '(':
                i += 1
                if has_select_keyword(content, i):
                    end_pos, nested_content, is_subquery = extract_subquery_content(content, i)
                    i = end_pos
                    
                    if is_subquery and nested_content and 'SELECT' in nested_content.upper():
                        # Check if this nested subquery is already in our list
                        is_new = True
                        for _, existing_content in subqueries + additional_subqueries:
                            # Use normalized comparison to avoid false duplicates
                            if nested_content.strip().lower() == existing_content.strip().lower():
                                is_new = False
                                break
                                
                        if is_new:
                            additional_subqueries.append(('Nested subquery', nested_content))
                else:
                    # Skip non-subquery parentheses
                    paren_count = 1
                    while i < len(content) and paren_count > 0:
                        if content[i] == '(':
                            paren_count += 1
                        elif content[i] == ')':
                            paren_count -= 1
                        i += 1
            else:
                i += 1
    
    # Add any additional nested subqueries found
    subqueries.extend(additional_subqueries)
    
    # Remove duplicates while preserving order
    seen = set()
    unique_subqueries = []
    for sq_type, sq_content in subqueries:
        if sq_content not in seen:
            seen.add(sq_content)
            unique_subqueries.append((sq_type, sq_content))
    
    return unique_subqueries

In [9]:
def generate_line_comparison_html(associations):
    """
    Generate HTML for line-by-line comparison
    
    Parameters:
    -----------
    associations : list
        List of dictionaries with matched lines and similarity scores
        
    Returns:
    --------
    str
        HTML string for the comparison visualization
    """
    # Add debug information
    print(f"Generating HTML for {len(associations)} associations")
    
    if not associations:
        return '<div class="error-container">No line associations to display.</div>'
    
    html_parts = ['<table class="line-comparison-table">']    
    html_parts.append('<tr><th>Line Type</th><th>Gold Query</th><th>Similarity</th><th>Predicted Query</th></tr>')
    
    for assoc in associations:
        gold_content = assoc.get('gold_content', "")
        pred_content = assoc.get('pred_content', "")
        line_type = assoc.get('gold_type', assoc.get('pred_type', "Unknown"))
        similarity = assoc.get('similarity', 0)
        
        # Format similarity for display
        similarity_display = f"{similarity:.2f}" if similarity > 0 else "-"
        
        # Determine cell classes based on matching status
        gold_class = "matched-content" if similarity > 0 else "unmatched-content" if gold_content else "missing-content"
        pred_class = "matched-content" if similarity > 0 else "unmatched-content" if pred_content else "missing-content"
        
        # Row opening
        html_parts.append(f'<tr class="{"matched-line" if similarity > 0 else ""}">') 
        
        # Line type cell
        html_parts.append(f'<td class="line-type">{line_type}</td>')
        
        # Gold content cell
        if gold_content:
            html_parts.append(f'<td class="gold-content {gold_class}">{gold_content}</td>')
        else:
            html_parts.append(f'<td class="gold-content missing-line">No matching line</td>')
        
        # Similarity cell
        html_parts.append(f'<td class="similarity">{similarity_display}</td>')
        
        # Predicted content cell
        if pred_content:
            html_parts.append(f'<td class="pred-content {pred_class}">{pred_content}</td>')
        else:
            html_parts.append(f'<td class="pred-content missing-line">No matching line</td>')
        
        # Row closing
        html_parts.append('</tr>')
    
    html_parts.append('</table>')
    return ''.join(html_parts)

In [10]:
class SQLLineComparisonWidget:
    """
    An interactive widget for visualizing and comparing SQL query predictions
    line by line with the gold queries in a Text-to-SQL task.
    """
    
    def __init__(self, results_dict, df_train=None, df_test=None, similarity_threshold=0.7):
        """
        Initialize the widget with a dictionary of result dictionaries.
        
        Parameters:
        -----------
        results_dict : dict
            Dictionary where keys are 'model_name-experiment_name' and values are lists of result dictionaries
        df_train : pandas.DataFrame, optional
            DataFrame containing training data with requests and knowledge
        df_test : pandas.DataFrame, optional
            DataFrame containing test data with requests and knowledge
        similarity_threshold : float
            Threshold for similarity to consider lines as associated
        """
        self.results_dict = results_dict
        self.df_train = df_train
        self.df_test = df_test
        self.similarity_threshold = similarity_threshold
        
        # Extract model and experiment data from the keys
        self.model_experiment_map = {}
        for key in results_dict.keys():
            if '-' in key:
                model, experiment = key.split('-', 1)
                if model not in self.model_experiment_map:
                    self.model_experiment_map[model] = []
                self.model_experiment_map[model].append(experiment)
        
        # Sort the experiments for each model
        for model in self.model_experiment_map:
            self.model_experiment_map[model] = sorted(self.model_experiment_map[model])
        
        # Flatten results for initial processing
        self.all_results = []
        for result_list in results_dict.values():
            self.all_results.extend(result_list)
        
        # Extract unique request IDs
        self.req_ids = sorted(list(set([r['req_id'] for r in self.all_results])))
        
        # Extract unique models and experiments
        self.models = sorted(list(self.model_experiment_map.keys()))
        if not self.models:
            self.models = ['All']
        
        # Extract unique difficulties
        self.difficulties = sorted(list(set([r.get('difficulty', 'unknown') for r in self.all_results if 'difficulty' in r])))
        self.difficulties = ['All'] + self.difficulties
        
        # Set up CSS styles for the UI
        self.setup_styles()
        
        # Create widgets
        self.create_widgets()
        
        # Connect event handlers
        self.setup_event_handlers()
        
        # Display the widget
        self.display_widget()
    
    def setup_styles(self):
        """Define CSS styles for the UI components."""
        self.style_html = """
        <style>
            /* Base container styles */
            .widget-container {
                max-width: 100%;
                margin: 10px 0;
                font-family: Arial, sans-serif;
            }
            
            /* Control panel styles */
            .control-panel {
                display: flex;
                flex-wrap: wrap;
                gap: 10px;
                margin-bottom: 15px;
                background-color: #f5f5f5;
                padding: 10px;
                border-radius: 5px;
            }
            
            /* Line comparison table styles */
            .line-comparison-table {
                width: 100%;
                border-collapse: collapse;
                margin: 10px 0;
                font-family: monospace;
                table-layout: fixed; /* Add fixed table layout for better control */
            }
            
            .line-comparison-table th {
                background-color: #f2f2f2;
                padding: 8px;
                text-align: left;
                border: 1px solid #ddd;
                position: sticky;
                top: 0;
                z-index: 10;
            }
            
            .line-comparison-table td {
                padding: 8px;
                border: 1px solid #ddd;
                vertical-align: top;
                word-wrap: break-word; /* Enable word wrapping */
                overflow-wrap: break-word; /* Modern browsers */
                word-break: break-word; /* For better line breaks */
                max-width: 0; /* Force text wrapping */
            }
            
            .line-type {
                font-weight: bold;
                width: 15%;
                background-color: #f8f8f8;
            }
            
            .gold-content, .pred-content {
                width: 38%; /* Slightly smaller to accommodate other columns */
                white-space: pre-wrap;
                overflow-x: auto; /* Add horizontal scrolling if needed */
            }
            
            .similarity {
                width: 9%; /* Fixed width for similarity column */
                text-align: center;
            }
                width: 5%;
                text-align: center;
            }
            
            /* Line matching styles */
            .matched-line {
                background-color: #f9f9f9;
            }
            
            .matched-content {
                background-color: #e6ffed;
                border-left: 3px solid #22863a;
            }
            
            .unmatched-content {
                background-color: #ffeef0;
                border-left: 3px solid #cb2431;
            }
            
            .missing-content {
                background-color: #f8f8f8;
            }
            
            /* Missing line indicator */
            .missing-line {
                color: #cb2431;
                font-style: italic;
            }
            
            /* Metadata container styles */
            .metadata-container {
                background-color: #e3f2fd;
                padding: 10px;
                border-radius: 5px;
                margin: 10px 0;
            }
            
            /* Request container styles */
            .request-container {
                background-color: #e8eaf6;
                padding: 10px;
                border-radius: 5px;
                margin: 10px 0;
                border-left: 4px solid #3f51b5;
            }
            
            /* Knowledge container styles */
            .knowledge-container {
                background-color: #f3e5f5;
                padding: 10px;
                border-radius: 5px;
                margin: 10px 0;
                border-left: 4px solid #9c27b0;
            }
            
            /* Summary container styles */
            .summary-container {
                background-color: #fff8e1;
                padding: 10px;
                border-radius: 5px;
                margin: 10px 0;
                border: 1px solid #ffe082;
                overflow-x: auto; /* Add horizontal scrolling for wide content */
                word-wrap: break-word; /* Enable word wrapping */
                overflow-wrap: break-word; /* For modern browsers */
            }
            
            /* Make sure pre elements in summary containers don't overflow */
            .summary-container pre {
                white-space: pre-wrap; /* Preserve formatting but wrap text */
                word-break: break-word; /* Break words to prevent overflow */
                max-width: 100%; /* Ensure content stays within container */
            }
            
            /* Style for the flex layout in raw queries */
            .summary-container .flex-container {
                display: flex;
                flex-wrap: wrap; /* Allow wrapping on smaller screens */
                gap: 20px;
            }
            
            .summary-container .flex-item {
                flex: 1 1 300px; /* Grow, shrink, and minimum width */
                min-width: 0; /* Allow flex items to shrink below content size */
            }
            
            /* Error container styles */
            .error-container {
                background-color: #ffebee;
                color: #c62828;
                padding: 10px;
                border-radius: 5px;
                margin: 10px 0;
                border-left: 4px solid #c62828;
            }
            
            /* Success container styles */
            .success-container {
                background-color: #e8f5e9;
                color: #2e7d32;
                padding: 10px;
                border-radius: 5px;
                margin: 10px 0;
                border-left: 4px solid #2e7d32;
            }
            
            /* Similarity slider container */
            .similarity-slider-container {
                margin: 10px 0;
                padding: 10px;
                background-color: #f0f0f0;
                border-radius: 5px;
            }
        </style>
        """
        display(HTML(self.style_html))
    
    def create_widgets(self):
        """Create the widgets for the UI."""
        # Model selector
        self.model_dropdown = widgets.Dropdown(
            options=self.models,
            value=self.models[0] if self.models else None,
            description='Model:',
            disabled=False if self.models and self.models[0] != 'All' else True,
            layout=widgets.Layout(width='300px')
        )
        
        # Experiment selector
        self.experiment_dropdown = widgets.Dropdown(
            description='Experiment:',
            disabled=False if self.model_experiment_map else True,
            layout=widgets.Layout(width='300px')
        )
        
        # Update experiment options based on model
        if self.model_experiment_map and self.models and self.models[0] != 'All':
            self.experiment_dropdown.options = ['All'] + self.model_experiment_map.get(self.models[0], [])
            self.experiment_dropdown.value = 'All'
        
        # Difficulty selector
        self.difficulty_dropdown = widgets.Dropdown(
            options=self.difficulties,
            value='All',
            description='Difficulty:',
            disabled=False,
            layout=widgets.Layout(width='200px')
        )
        
        # Request ID selector
        self.req_id_dropdown = widgets.Dropdown(
            options=self.req_ids,
            description='Request ID:',
            disabled=False,
            layout=widgets.Layout(width='300px')
        )
        
        # Search box
        self.search_box = widgets.Text(
            value='',
            placeholder='Search queries...',
            description='Search:',
            disabled=False,
            layout=widgets.Layout(width='300px')
        )
        
        # Search button
        self.search_button = widgets.Button(
            description='Search',
            disabled=False,
            button_style='info',
            tooltip='Search for queries containing the text',
            icon='search'
        )
        
        # Similarity method selector
        self.similarity_method_dropdown = widgets.Dropdown(
            options=['tfidf', 'jaccard', 'embedding'],
            value='tfidf',
            description='Similarity Method:',
            disabled=False,
            layout=widgets.Layout(width='200px')
        )
        
        # Similarity threshold slider
        self.similarity_slider = widgets.FloatSlider(
            value=self.similarity_threshold,
            min=0.0,
            max=1.0,
            step=0.05,
            description='Similarity Threshold:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            layout=widgets.Layout(width='400px')
        )
        
        # Update button
        self.update_button = widgets.Button(
            description='Update Comparison',
            disabled=False,
            button_style='primary',
            tooltip='Update the comparison with new threshold',
            icon='refresh'
        )
        
        # Show raw queries toggle
        self.show_raw_queries_toggle = widgets.Checkbox(
            value=True,
            description='Show Raw Queries',
            disabled=False,
            indent=False,
            layout=widgets.Layout(width='200px')
        )
        
        # Results display
        self.output = widgets.Output()
    
    def setup_event_handlers(self):
        """Connect the widget event handlers."""
        # When model changes, update experiments
        self.model_dropdown.observe(self.on_model_change, names='value')
        
        # When experiment, difficulty, or request ID changes, update display
        self.experiment_dropdown.observe(self.on_params_change, names='value')
        self.difficulty_dropdown.observe(self.on_params_change, names='value')
        self.req_id_dropdown.observe(self.on_req_id_change, names='value')
        
        # When search button is clicked, perform search
        self.search_button.on_click(self.on_search)
        self.search_box.on_submit(self.on_search)
        
        # When update button is clicked, refresh the comparison
        self.update_button.on_click(self.on_update)
        
        # When similarity method changes, update the display
        self.similarity_method_dropdown.observe(self.on_similarity_method_change, names='value')
        
        # When show raw queries toggle changes, update the display
        self.show_raw_queries_toggle.observe(self.on_toggle_raw_queries, names='value')
    
    def on_model_change(self, change):
        """Handle model selection change."""
        model = change['new']
        if model and model != 'All' and model in self.model_experiment_map:
            self.experiment_dropdown.options = ['All'] + self.model_experiment_map[model]
            self.experiment_dropdown.value = 'All'
        else:
            self.experiment_dropdown.options = ['All']
            self.experiment_dropdown.value = 'All'
    
    def on_params_change(self, change):
        """Handle parameter changes (model, experiment, difficulty)."""
        self.filter_req_ids()
    
    def on_req_id_change(self, change):
        """Handle request ID selection change."""
        self.update_display()
    
    def on_search(self, button=None):
        """Handle search action."""
        search_text = self.search_box.value.lower()
        if not search_text:
            self.filter_req_ids()
            return
        
        filtered_req_ids = []
        for result_list in self.results_dict.values():
            for result in result_list:
                req_id = result['req_id']
                
                # Get gold and predicted queries
                if 'comparison' in result and isinstance(result['comparison'], dict):
                    comparison = result['comparison']
                    gold_query = comparison.get('gold_query', '')
                    pred_query = comparison.get('pred_query', '')
                    
                    # Check if search text is in either query
                    if (search_text in gold_query.lower() or 
                        search_text in pred_query.lower()):
                        filtered_req_ids.append(req_id)
        
        # Update the request ID dropdown
        filtered_req_ids = sorted(list(set(filtered_req_ids)))
        if filtered_req_ids:
            self.req_id_dropdown.options = filtered_req_ids
            self.req_id_dropdown.value = filtered_req_ids[0]
        else:
            self.req_id_dropdown.options = ['No results']
            self.req_id_dropdown.value = 'No results'
    
    def on_update(self, button=None):
        """Handle update button click."""
        self.similarity_threshold = self.similarity_slider.value
        self.update_display()
    
    def on_similarity_method_change(self, change):
        """Handle change in similarity method."""
        if change['new'] == 'embedding':
            # Ensure the embedding model is initialized
            initialize_sentence_model()
        self.update_display()
    
    def on_toggle_raw_queries(self, change):
        """Handle toggle for showing raw queries."""
        self.update_display()
    
    def filter_req_ids(self):
        """Filter request IDs based on selected model, experiment, and difficulty."""
        model = self.model_dropdown.value
        experiment = self.experiment_dropdown.value
        difficulty = self.difficulty_dropdown.value
        
        filtered_req_ids = []
        
        for key, result_list in self.results_dict.items():
            # Check if this result matches the selected model and experiment
            if model != 'All' and not key.startswith(f"{model}-"):
                continue
            if experiment != 'All' and not key.endswith(f"-{experiment}"):
                continue
            
            # Add matching request IDs
            for result in result_list:
                if difficulty != 'All' and result.get('difficulty') != difficulty:
                    continue
                filtered_req_ids.append(result['req_id'])
        
        # Update the request ID dropdown
        filtered_req_ids = sorted(list(set(filtered_req_ids)))
        if filtered_req_ids:
            self.req_id_dropdown.options = filtered_req_ids
            if self.req_id_dropdown.value not in filtered_req_ids:
                self.req_id_dropdown.value = filtered_req_ids[0]
        else:
            self.req_id_dropdown.options = ['No results']
            self.req_id_dropdown.value = 'No results'
    
    def update_display(self):
        """Update the display with the selected request ID, model, and experiment."""
        req_id = self.req_id_dropdown.value
        model = self.model_dropdown.value
        experiment = self.experiment_dropdown.value

        if req_id == 'No results':
            with self.output:
                clear_output()
                display(HTML('<div class="error-container">No results found.</div>'))
            return
        
        # Find result(s) matching the request ID
        matching_results = []
        for key, result_list in self.results_dict.items():
            for result in result_list:
                if result['req_id'] == req_id and key == f"{model}-{experiment}":
                    result['model_experiment'] = key
                    matching_results.append(result)
        
        if not matching_results:
            with self.output:
                clear_output()
                display(HTML('<div class="error-container">No matching result found.</div>'))
            return
        
        # Sort results by model and experiment
        matching_results.sort(key=lambda r: r['model_experiment'])
        
        # Display the results
        with self.output:
            clear_output()
            
            # If multiple results, allow selection
            # if len(matching_results) > 1:
            #     result_dropdown = widgets.Dropdown(
            #         options=[(f"{r['model_experiment']}", i) for i, r in enumerate(matching_results)],
            #         description='Result:',
            #         layout=widgets.Layout(width='400px')
            #     )
                
            #     def on_result_change(change):
            #         self.display_single_result(matching_results[change['new']])
                
            #     result_dropdown.observe(on_result_change, names='value')
            #     display(result_dropdown)
            #     self.display_single_result(matching_results[0])
            # else:
            print(matching_results)
            self.display_single_result(matching_results[0])
    
    def display_single_result(self, result):
        """Display a single result with line-by-line comparison."""
        if 'comparison' not in result or not isinstance(result['comparison'], dict):
            display(HTML('<div class="error-container">No comparison data found.</div>'))
            return
        
        comparison = result['comparison']
        gold_query = comparison.get('sql_gold', '')
        pred_query = comparison.get('sql_pred', '')
        
        # Debug info about the queries
        print(f"Gold query type: {type(gold_query)}, length: {len(str(gold_query)) if gold_query else 0}")
        print(f"Predicted query type: {type(pred_query)}, length: {len(str(pred_query)) if pred_query else 0}")
        
        # Ensure queries are strings and clean them up
        if gold_query is None:
            gold_query = ""
        elif not isinstance(gold_query, str):
            gold_query = str(gold_query)
            
        if pred_query is None:
            pred_query = ""
        elif not isinstance(pred_query, str):
            pred_query = str(pred_query)
        
        # Remove extra quotes that might be present in the queries
        gold_query = gold_query.strip().strip('"\'')
        pred_query = pred_query.strip().strip('"\'')
        
        # Extract subqueries and nested queries from the gold and predicted queries
        gold_subqueries = extract_subqueries(gold_query)
        pred_subqueries = extract_subqueries(pred_query)

        # Parse queries into lines
        gold_lines = parse_sql_into_lines(gold_query)
        pred_lines = parse_sql_into_lines(pred_query)

        # Debug info about parsed lines
        print(f"Gold lines: {gold_lines}")
        print(f"Predicted lines: {pred_lines}")
        
        # Get the selected similarity method
        similarity_method = self.similarity_method_dropdown.value
        print(f"Using similarity method: {similarity_method}")
        
        # If using embedding method, initialize the model
        if similarity_method == 'embedding' and sentence_model is None:
            initialize_sentence_model()
        
        # Associate lines with the selected method
        associations = []
        
        # Keep track of which lines have been matched
        gold_matched = [False] * len(gold_lines)
        pred_matched = [False] * len(pred_lines)
        
        # Calculate similarity matrix
        similarity_matrix = np.zeros((len(gold_lines), len(pred_lines)))
        
        for i, (g_type, g_content) in enumerate(gold_lines):
            for j, (p_type, p_content) in enumerate(pred_lines):
                # First, try to match lines of the same type
                if g_type == p_type:
                    similarity = calculate_similarity_with_method(g_content, p_content, similarity_method)
                    similarity_matrix[i, j] = similarity
                else:
                    # If types don't match but both are WHERE conditions or similar, still try to match
                    if ('WHERE' in g_type and 'WHERE' in p_type) or \
                       ('SELECT' in g_type and 'SELECT' in p_type) or \
                       ('FROM' in g_type and 'FROM' in p_type) or \
                       ('JOIN' in g_type and 'JOIN' in p_type):
                        # Reduce similarity slightly for different subtypes
                        similarity = calculate_similarity_with_method(g_content, p_content, similarity_method) * 0.9
                        similarity_matrix[i, j] = similarity
                    else:
                        # similarity_matrix[i, j] = 0
                        similarity = calculate_similarity_with_method(g_content, p_content, similarity_method) * 0.75
                        similarity_matrix[i, j] = similarity
        
        # Associate lines greedily from highest to lowest similarity
        while np.max(similarity_matrix) > self.similarity_threshold:
            # Find the highest similarity
            max_i, max_j = np.unravel_index(np.argmax(similarity_matrix), similarity_matrix.shape)
            max_sim = similarity_matrix[max_i, max_j]
            
            # Add the association
            associations.append({
                'gold_index': max_i,
                'gold_type': gold_lines[max_i][0],
                'gold_content': gold_lines[max_i][1],
                'pred_index': max_j,
                'pred_type': pred_lines[max_j][0],
                'pred_content': pred_lines[max_j][1],
                'similarity': max_sim
            })
            
            # Mark as matched
            gold_matched[max_i] = True
            pred_matched[max_j] = True
            
            # Remove these lines from consideration
            similarity_matrix[max_i, :] = 0
            similarity_matrix[:, max_j] = 0
        
        # Add unmatched gold lines
        for i, (g_type, g_content) in enumerate(gold_lines):
            if not gold_matched[i]:
                associations.append({
                    'gold_index': i,
                    'gold_type': g_type,
                    'gold_content': g_content,
                    'pred_index': None,
                    'pred_type': None,
                    'pred_content': None,
                    'similarity': 0
                })
        
        # Add unmatched predicted lines
        for j, (p_type, p_content) in enumerate(pred_lines):
            if not pred_matched[j]:
                associations.append({
                    'gold_index': None,
                    'gold_type': None,
                    'gold_content': None,
                    'pred_index': j,
                    'pred_type': p_type,
                    'pred_content': p_content,
                    'similarity': 0
                })
        
        # Sort by gold index, then pred index
        associations.sort(key=lambda x: (
            float('inf') if x['gold_index'] is None else x['gold_index'], 
            float('inf') if x['pred_index'] is None else x['pred_index']
        ))
        
        # Generate HTML
        line_comparison_html = generate_line_comparison_html(associations)
        
        # Get request and knowledge information
        req_id = result['req_id']
        request_text = ''
        knowledge_text = ''
        
        # Look up in train and test data
        for df in [self.df_train, self.df_test]:
            if df is not None:
                try:
                    # Try different ways of matching the ID
                    # First, print column names for debugging
                    print(f"DataFrame columns: {df.columns.tolist()}")
                    
                    # Check if 'id' or 'req_id' column exists
                    id_col = None
                    for possible_id_col in ['id', 'req_id', 'ID', 'Id']:
                        if possible_id_col in df.columns:
                            id_col = possible_id_col
                            break
                    
                    if id_col is not None:
                        # Convert req_id to appropriate type for comparison
                        try:
                            req_id_value = int(req_id)
                        except (ValueError, TypeError):
                            req_id_value = req_id
                            
                        # Try to find matching rows
                        matching_rows = df[df[id_col] == req_id_value]
                        
                        # If no match with numerical comparison, try string comparison
                        if matching_rows.empty and isinstance(req_id_value, int):
                            matching_rows = df[df[id_col] == str(req_id_value)]
                        
                        if not matching_rows.empty:
                            row = matching_rows.iloc[0]
                            print(f"Found matching row with ID {req_id_value}")
                            
                            # Check for request column with different possible names
                            for req_col in ['request', 'question', 'query', 'text']:
                                if req_col in row:
                                    request_text = row[req_col]
                                    break
                            
                            # Check for knowledge column with different possible names
                            for know_col in ['external_knowledge', 'knowledge', 'schema', 'context']:
                                if know_col in row:
                                    knowledge_text = row[know_col]
                                    break
                            for dom_know_col in ['domain_knowledge', 'knowledge', 'schema', 'context']:
                                if dom_know_col in row:
                                    knowledge_text += f"\n {row[dom_know_col]}"
                                    break
                            
                            break  # Break out of the df loop if we found a match
                except Exception as e:
                    print(f"Error looking up request info: {e}")
        
        # Display metadata
        metadata_html = f"""
        <div class="metadata-container">
            <h3>Metadata</h3>
            <p><strong>Request ID:</strong> {req_id}</p>
            <p><strong>Model:</strong> {result['model_experiment'].split('-')[0]}</p>
            <p><strong>Experiment:</strong> {'-'.join(result['model_experiment'].split('-')[1:])}</p>
            <p><strong>Difficulty:</strong> {result.get('difficulty', 'Unknown')}</p>
        </div>
        """
        
        # Display request and knowledge
        request_html = f"""
        <div class="request-container">
            <h3>Request</h3>
            <p>{request_text}</p>
        </div>
        """
        
        knowledge_html = f"""
        <div class="knowledge-container">
            <h3>External Knowledge</h3>
            <p>{knowledge_text}</p>
        </div>
        """
        
        # Display raw queries for comparison/debugging
        raw_queries_html = f"""
        <div class="summary-container">
            <h3>Raw Queries</h3>
            <div class="flex-container">
                <div class="flex-item">
                    <h4>Gold Query</h4>
                    <pre style="background-color: #f5f5f5; padding: 10px; overflow-x: auto;">{gold_query}</pre>
                </div>
                <div class="flex-item">
                    <h4>Predicted Query</h4>
                    <pre style="background-color: #f5f5f5; padding: 10px; overflow-x: auto;">{pred_query}</pre>
                </div>
            </div>
        </div>
        """
        
        # Display summary statistics
        matched_count = sum(1 for a in associations if a['similarity'] > 0)
        total_gold = sum(1 for a in associations if a['gold_index'] is not None)
        total_pred = sum(1 for a in associations if a['pred_index'] is not None)
        
        # Avoid division by zero
        gold_coverage = matched_count/max(total_gold, 1) if total_gold > 0 else 0
        pred_coverage = matched_count/max(total_pred, 1) if total_pred > 0 else 0
        
        summary_html = f"""
        <div class="summary-container">
            <h3>Comparison Summary</h3>
            <p><strong>Matched Lines:</strong> {matched_count}</p>
            <p><strong>Gold Lines:</strong> {total_gold}</p>
            <p><strong>Predicted Lines:</strong> {total_pred}</p>
            <p><strong>Gold Lines:</strong> {gold_lines}</p>
            <p><strong>Predicted Lines:</strong> {pred_lines}</p>        
            <p><strong>Gold Coverage:</strong> {gold_coverage:.2f} ({matched_count}/{total_gold})</p>
            <p><strong>Prediction Coverage:</strong> {pred_coverage:.2f} ({matched_count}/{total_pred})</p>
        </div>
        """
        
        # Display similarity threshold slider in the results area
        similarity_slider_html = f"""
        <div class="similarity-slider-container">
            <h3>Adjust Similarity Threshold</h3>
            <p>Current threshold: {self.similarity_threshold:.2f}</p>
            <p>Higher values require more similarity for lines to match. Lower values are more lenient in matching.</p>
        </div>
        """
        
        # Display execution information if available
        execution_html = ""
        if 'execution_time' in comparison or 'execution_error' in comparison:
            execution_status = "Failed" if comparison.get('execution_error') else "Succeeded"
            execution_time = comparison.get('execution_time', 'N/A')
            execution_error = comparison.get('execution_error', 'None')
            
            execution_html = f"""
            <div class="{'error-container' if execution_status == 'Failed' else 'success-container'}">
                <h3>Execution Status: {execution_status}</h3>
                <p><strong>Execution Time:</strong> {execution_time}</p>
                {f'<p><strong>Error:</strong> {execution_error}</p>' if execution_status == 'Failed' else ''}
            </div>
            """
        
        # Combine all HTML
        final_html = f"""
        <div class="widget-container">
            {metadata_html}
            {request_html}
            {knowledge_html}
            {raw_queries_html if self.show_raw_queries_toggle.value else ''}
            {summary_html}
            {similarity_slider_html}
            {execution_html}
            <h3>Line-by-Line Comparison</h3>
            {line_comparison_html}
        </div>
        """
        
        display(HTML(final_html))
    
    def display_widget(self):
        """Display the complete widget."""
        # Create the control panel
        control_panel = widgets.VBox([
            widgets.HBox([
                self.model_dropdown,
                self.experiment_dropdown,
                self.difficulty_dropdown
            ]),
            widgets.HBox([
                self.req_id_dropdown,
                self.search_box,
                self.search_button
            ]),
            widgets.HBox([
                self.similarity_method_dropdown,
                self.similarity_slider,
                self.update_button
            ]),
            widgets.HBox([
                self.show_raw_queries_toggle
            ])
        ])
        
        # Display the control panel and output
        display(HTML('<h2>SQL Line-by-Line Comparison Widget</h2>'))
        display(HTML('<div class="control-panel">'))
        display(control_panel)
        display(HTML('</div>'))
        display(self.output)
        
        # Initialize the display
        self.filter_req_ids()

In [11]:
# Create and display the widget
widget = SQLLineComparisonWidget(results_dict, db_train, db_test, similarity_threshold=0.7)

Output()

## About the SQL Line-by-Line Comparison Widget

This widget provides a specialized visualization for comparing SQL queries line by line, focusing on associating similar clauses, conditions, and statements between gold (reference) and predicted SQL queries.

### Key Features

1. **Line Parsing**: Breaks down SQL queries into meaningful "lines" (clauses, conditions, statements) that can be compared individually.

2. **Similarity Calculation**: Uses multiple similarity methods to determine how similar each line from the gold query is to each line from the predicted query:
   - TF-IDF vectorization and cosine similarity (default)
   - Jaccard similarity (set-based comparison)
   - Word embeddings (neural semantic similarity)
   
   See `similarity_methods_documentation.md` for detailed information on each method.

3. **Line Association**: Associates lines from both queries based on a configurable similarity threshold.

4. **Visual Highlighting**: 
   - Lines that match (above the similarity threshold) are highlighted in green
   - Lines that don't have a match in the other query are highlighted in red

5. **Adjustable Threshold**: The similarity threshold can be adjusted to control how strict the matching should be.

6. **Summary Statistics**: Provides metrics on how many lines matched, total lines in each query, and coverage percentages.

7. **Filtering Options**: Filter by model, experiment, difficulty level, or search for specific SQL content.

### Benefits

- Provides a more granular comparison than simple query-level matching
- Helps identify which specific parts of queries match or differ
- Allows for understanding partial correctness in predicted queries
- Supports detailed error analysis and model comparison

## Enhanced Subquery Handling

The SQL Line Comparison Widget now includes improved handling for subqueries and nested queries with the following enhancements:

### Dedicated Subquery Extraction
- A new `extract_subqueries` function identifies and extracts subqueries from different parts of the SQL statement:
  - SELECT clause subqueries (scalar subqueries)
  - FROM clause subqueries (derived tables)
  - WHERE clause subqueries (IN, EXISTS, comparison)
  - JOIN condition subqueries

### Subquery-Aware Parsing Process
1. Subqueries are first extracted from the original SQL
2. Placeholder text replaces subqueries in the main query
3. The modified query is parsed normally
4. Placeholders are replaced with the original subqueries when building the line items
5. Special line types are used to distinguish items containing subqueries

### New Line Types for Subqueries
- `SELECT item with subquery` - For columns that use subqueries
- `FROM subquery` - For derived tables and table expressions
- `WHERE condition with subquery` - For WHERE conditions that use subqueries
- `JOIN subquery` - For JOIN clauses that use subqueries
- `Independent [type] subquery` - For subqueries that weren't matched to a specific clause

### Benefits
- Preserves the structure of complex queries
- Keeps subqueries intact for better comparison
- Properly identifies and matches similar subqueries between gold and predicted SQL
- Handles nested levels of subqueries
- Improves similarity matching for queries with complex structures

This enhancement significantly improves the widget's ability to handle complex SQL queries with nested structures, making the line-by-line comparison more accurate and useful for advanced SQL evaluation.